# Vogue Runway - Webscraping

## 

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import requests
import os
import time
from PIL import Image
from bs4 import BeautifulSoup  # Import BeautifulSoup

In [2]:
# Set up Chrome options
options = Options()
#options.add_argument('--headless')  
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [3]:
# Initialize ChromeDriver
service = Service('/opt/homebrew/Caskroom/chromedriver/127.0.6533.72/chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=service, options=options)

In [4]:
def login_to_vogue(email, password):
    # Open the login page
    login_url = "https://id.condenast.com/interaction/th29Mr4m8vZGCewhVVfB-/select-account?xid=9bec3edd-6a7a-4833-a26e-eb4951b3d28b&scope=openid%20offline_access&state=%7B%22redirectURL%22%3A%22%2Fnewsletter%2Frunway%22%7D&prompt=select_account%20consent&source=VERSO_NAVIGATION&client_id=condenast.identity.fbc9096dc61f9b79c5ac4c85998da075&redirect_uri=https%3A%2F%2Fwww.vogue.com%2Fauth%2Fcomplete&response_type=code"  
    driver.get(login_url)
    
    wait = WebDriverWait(driver, 20)
    
    # Debugging: Print page source to verify if the login fields are present
    print(driver.page_source)
    
    username_field = wait.until(EC.presence_of_element_located((By.NAME, 'email')))
    username_field.send_keys(email)
    
    
    next_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="email-continue-button"]')))
    next_button.click()

    
    # Wait for the password field to be visible and fill in the password
    password_field = wait.until(EC.presence_of_element_located((By.ID, 'TextField-id-password')))
    password_field.send_keys(password)
    
    
    login_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="log-in-sign-in-button"]')))
    login_button.click()
    
    # Wait for the login to complete and redirect
    wait.until(EC.url_changes(login_url))  # Wait until the URL changes after login

In [5]:
email = "< enter registered email address here > "
password = "< password >"

In [6]:
# Log in to Vogue
login_to_vogue(email, password)

<html lang="en-US"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><meta http-equiv="x-ua-compatible" content="ie=edge"><meta http-equiv="Content-Security-Policy" content="default-src 'self' c.condenast.com qc.condenast.com ssc.condenast.com api.avo.app geolocation.onetrust.com *.quantummetric.com cdn.cookielaw.org www.gstatic.com www.google.com *.googletagmanager.com *.google-analytics.com *.permutive.com *.treasuredata.com condenast-privacy.my.onetrust.com 'unsafe-inline' 'unsafe-eval'; worker-src blob:; child-src * 'self' www.google.com www.gstatic.com; script-src 'self' c.condenast.com qc.condenast.com ssc.condenast.com globalservices.conde.digital cdn.cookielaw.org www.gstatic.com www.google.com *.googletagmanager.com *.google-analytics.com *.quantummetric.com cdn.avo.app *.permutive.com *.treasuredata.com 'unsafe-inline' 'unsafe-eval'; img-src * 'self'; "><title>Sign in or create an account</title><link rel="shortc

In [7]:
target_url = "https://www.vogue.com/fashion-shows/resort-2025/gucci/slideshow/collection#1"
driver.get(target_url)

In [8]:
print("Navigating to URL...")
driver.get(target_url)
print("Page title:", driver.title)

Navigating to URL...
Page title: Gucci Resort 2025 Fashion Show | Vogue


In [10]:
wait = WebDriverWait(driver, 120)  # Increase timeout
main_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#main-content > div')))
print("Main container found")

Main container found


In [11]:
# Collect image elements with sequential CSS selectors
image_elements = []
number_of_images = 45
#Create a directory to save images
save_path = "resort_2025_Gucci_images"
os.makedirs(save_path, exist_ok=True)

In [12]:
# Function to resize an image
def resize_image(image_path, max_size=(300, 300)):
    with Image.open(image_path) as img:
        img.thumbnail(max_size, Image.ANTIALIAS)
        img.save(image_path)

In [13]:
## ANOTHER NEW ONE ###

# Function to extract all image URLs from each set using BeautifulSoup
def get_image_urls_using_bs4(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    img_urls = []

    # Find all the list items which represent the sets of images
    image_sets = soup.select('li.ImageCollectionListItem-YjTJj.eOyRQu')  # This should match the sets

    for image_set in image_sets:
        # Select all images in the current set
        img_elements = image_set.select('div.ZoomableWrapper-kJXWeT img')  # Select all images in the set
        for img_element in img_elements:
            # Extract URL from srcset or src
            img_url = img_element.get('srcset') or img_element.get('src')
            if img_url:
                # Clean the URL if it's in srcset format (e.g., select the highest resolution URL)
                img_url = img_url.split(',')[-1].strip().split(' ')[0] if ' ' in img_url else img_url
                img_urls.append(img_url)
    
    print(f"Extracted image URLs: {img_urls}")  # Debug line to check extracted URLs
    return img_urls

In [14]:
# Function to click the "Next" button
def click_next_button():
    wait = WebDriverWait(driver, 30)  # Increase timeout if necessary
    try:
        print("Attempting to find the 'Next' button...")
        next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main-content > div.RunwayDesktopGalleryWrapper-jHDjjy.czoxGK > div.GridWrapper-cAzTTK.jXvsWG.grid.grid-margins.grid-items-0.RunwayGalleryGridWrapper-cFBsPI.eVwxQs > div:nth-child(1) > div > div > div.StickyBoxPrimary-dzWDWL.cdhYoN.sticky-box__primary > div.RunwayGalleryLookNumberWrapper-dIlzYs.iJWreG > div:nth-child(3)')))
        print("Found the 'Next' button. Clicking...")
        next_button.click()
        print("Clicked the 'Next' button successfully.")
        
        # Wait longer to ensure the new images load
        time.sleep(10)  # Increase if needed

        # Optionally, add a scroll here
        scroll_vertically()

    except TimeoutException:
        print("Timeout: 'Next' button not clickable or not found.")
    except Exception as e:
        print(f"Failed to click the 'Next' button: {e}")


In [15]:
#vertical scrolling to scroll through both the images under the main image
def scroll_vertically():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

In [16]:
def wait_for_new_images_to_load():
    # Wait for an element that signifies new images are loaded or content has changed
    try:
        WebDriverWait(driver, 40).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[2]/div[1]/div[2]/div/div/ul/li[1]/div[1]/div/span/picture/img")))
        print("New images loaded.")
    except TimeoutException:
        print("Timeout: New images did not load.")
        

In [17]:
# Accumulate image URLs across all sets
all_image_urls = set()  # Use a set to avoid duplicates
downloaded_images = 0
max_images = float('inf')  # To download all available images
seen_image_urls = []  # To keep track of the last few sets of URLs
repeat_threshold = 2  # Number of repeats before breaking loop
consecutive_repeats = 0  # Counter for consecutive repeats

while downloaded_images < max_images:
    # Get the page source and parse it with BeautifulSoup
    page_source = driver.page_source
    image_urls = get_image_urls_using_bs4(page_source)

    # Check if new URLs were found
    if image_urls:
        unique_image_urls = set(image_urls)
        if unique_image_urls and not unique_image_urls.issubset(all_image_urls):
            all_image_urls.update(unique_image_urls)
            downloaded_images += len(unique_image_urls)  # Update count
            
            # Update the seen image sets
            seen_image_urls.append(unique_image_urls)
            if len(seen_image_urls) > 3:  # Keep only the last three sets
                seen_image_urls.pop(0)
            
            # Check if the current set matches any of the last two sets
            if seen_image_urls.count(unique_image_urls) > repeat_threshold:
                print("Repeated the same set of images multiple times. Stopping.")
                break
            
            # Click the "Next" button to navigate to the next set of images
            click_next_button()
            wait_for_new_images_to_load()
        else:
            print("No new images found or repeated set.")
            consecutive_repeats += 1
            if consecutive_repeats >= repeat_threshold:
                print("Consecutive repeats threshold reached. Stopping.")
                break
    else:
        print("No image URLs found or no new images available.")
        break

# After collecting all URLs, download images once
print(f"Total unique images to download: {len(all_image_urls)}")

Extracted image URLs: ['https://assets.vogue.com/photos/664331c534b302fd2a217eb8/master/w_2560%2Cc_limit/00001-gucci-resort-2025-credit-brand.jpg', 'https://assets.vogue.com/photos/66437c5aa7c71495274d254a/master/w_2560%2Cc_limit/00001-gucci-resort-2025-details-credit-brand.jpg', 'https://assets.vogue.com/photos/66437c588f7b72a40928dc84/master/w_2560%2Cc_limit/00002-gucci-resort-2025-details-credit-brand.jpg']
Attempting to find the 'Next' button...
Found the 'Next' button. Clicking...
Clicked the 'Next' button successfully.
New images loaded.
Extracted image URLs: ['https://assets.vogue.com/photos/664331c65fcec4340d6cd63a/master/w_2560%2Cc_limit/00002-gucci-resort-2025-credit-brand.jpg', 'https://assets.vogue.com/photos/66437c5a3685f0feab7d00da/master/w_2560%2Cc_limit/00003-gucci-resort-2025-details-credit-brand.jpg', 'https://assets.vogue.com/photos/66437c5c79ab2b5c24ad0611/master/w_2560%2Cc_limit/00004-gucci-resort-2025-details-credit-brand.jpg']
Attempting to find the 'Next' button

Found the 'Next' button. Clicking...
Clicked the 'Next' button successfully.
New images loaded.
Extracted image URLs: ['https://assets.vogue.com/photos/664331d94f0fc69cad53317e/master/w_2560%2Cc_limit/00020-gucci-resort-2025-credit-brand.jpg', 'https://assets.vogue.com/photos/66437c7a8c081dabc8b7f33f/master/w_2560%2Cc_limit/00025-gucci-resort-2025-details-credit-brand.jpg']
Attempting to find the 'Next' button...
Found the 'Next' button. Clicking...
Clicked the 'Next' button successfully.
New images loaded.
Extracted image URLs: ['https://assets.vogue.com/photos/664331dbfb309270cd42e099/master/w_2560%2Cc_limit/00021-gucci-resort-2025-credit-brand.jpg', 'https://assets.vogue.com/photos/66437c7ba1ad1be29a88f3fa/master/w_2560%2Cc_limit/00026-gucci-resort-2025-details-credit-brand.jpg']
Attempting to find the 'Next' button...
Found the 'Next' button. Clicking...
Clicked the 'Next' button successfully.
New images loaded.
Extracted image URLs: ['https://assets.vogue.com/photos/664331dbefb7e8

Found the 'Next' button. Clicking...
Clicked the 'Next' button successfully.
New images loaded.
Extracted image URLs: ['https://assets.vogue.com/photos/664331efea2c6e6d9b713920/master/w_2560%2Cc_limit/00040-gucci-resort-2025-credit-brand.jpg', 'https://assets.vogue.com/photos/66437c9625ca0bbc6465c472/master/w_2560%2Cc_limit/00045-gucci-resort-2025-details-credit-brand.jpg']
Attempting to find the 'Next' button...
Found the 'Next' button. Clicking...
Clicked the 'Next' button successfully.
New images loaded.
Extracted image URLs: ['https://assets.vogue.com/photos/6643814a018807e7bb2bbc7d/master/w_2560%2Cc_limit/00040b-gucci-resort-2025-credit-brand.jpg']
Attempting to find the 'Next' button...
Found the 'Next' button. Clicking...
Clicked the 'Next' button successfully.
New images loaded.
Extracted image URLs: ['https://assets.vogue.com/photos/664331f06018dfacd8408cd3/master/w_2560%2Cc_limit/00041-gucci-resort-2025-credit-brand.jpg']
Attempting to find the 'Next' button...
Found the 'Nex

In [18]:
# Function to download images
def download_images(image_urls):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    for index, img_url in enumerate(image_urls):
        try:
            # Log the final image URL
            print(f"Final Image {index + 1} URL: {img_url}")

            # Validate the image URL
            if not img_url.startswith('http'):
                print(f"Invalid URL: {img_url}")
                continue

            # Define the image name and path
            img_name = f"image_{index + 1}.jpg"
            img_path = os.path.join(save_path, img_name)

            # Download the image
            response = requests.get(img_url, stream=True)
            response.raise_for_status()  # Check for HTTP request errors

            # Write the image to a file
            with open(img_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        file.write(chunk)
            
            print(f"Image {index + 1} downloaded successfully")

        except requests.RequestException as e:
            print(f"Failed to download image {index + 1} due to a request error: {e}")
        except IOError as e:
            print(f"Failed to write image {index + 1} to disk: {e}")
        except Exception as e:
            print(f"Failed to download image {index + 1} due to an unexpected error: {e}")

# Call the download function
download_images(list(all_image_urls))


Final Image 1 URL: https://assets.vogue.com/photos/66437ca2c5e6224618e42d6f/master/w_2560%2Cc_limit/00054-gucci-resort-2025-details-credit-brand.jpg
Image 1 downloaded successfully
Final Image 2 URL: https://assets.vogue.com/photos/664331f69a3c66facb58cbe8/master/w_2560%2Cc_limit/00046-gucci-resort-2025-credit-brand.jpg
Image 2 downloaded successfully
Final Image 3 URL: https://assets.vogue.com/photos/664331deb39367a055268ec5/master/w_2560%2Cc_limit/00024-gucci-resort-2025-credit-brand.jpg
Image 3 downloaded successfully
Final Image 4 URL: https://assets.vogue.com/photos/66437c80f1921afeeed663c8/master/w_2560%2Cc_limit/00030-gucci-resort-2025-details-credit-brand.jpg
Image 4 downloaded successfully
Final Image 5 URL: https://assets.vogue.com/photos/664331d16bcc40a0ed5ce464/master/w_2560%2Cc_limit/00012-gucci-resort-2025-credit-brand.jpg
Image 5 downloaded successfully
Final Image 6 URL: https://assets.vogue.com/photos/664331e60361c9f61abfb186/master/w_2560%2Cc_limit/00032-gucci-resort-

Image 47 downloaded successfully
Final Image 48 URL: https://assets.vogue.com/photos/664331d94f0fc69cad53317e/master/w_2560%2Cc_limit/00020-gucci-resort-2025-credit-brand.jpg
Image 48 downloaded successfully
Final Image 49 URL: https://assets.vogue.com/photos/66437c8d3fe44e5ea5973e1e/master/w_2560%2Cc_limit/00038-gucci-resort-2025-details-credit-brand.jpg
Image 49 downloaded successfully
Final Image 50 URL: https://assets.vogue.com/photos/664331cb0361c9f61abfb184/master/w_2560%2Cc_limit/00005-gucci-resort-2025-credit-brand.jpg
Image 50 downloaded successfully
Final Image 51 URL: https://assets.vogue.com/photos/664331dbefb7e824f0df4818/master/w_2560%2Cc_limit/00022-gucci-resort-2025-credit-brand.jpg
Image 51 downloaded successfully
Final Image 52 URL: https://assets.vogue.com/photos/664331e105ebfa01dfa627c5/master/w_2560%2Cc_limit/00027-gucci-resort-2025-credit-brand.jpg
Image 52 downloaded successfully
Final Image 53 URL: https://assets.vogue.com/photos/664331e5606e6cbbf54ba1e6/master/

Image 93 downloaded successfully
Final Image 94 URL: https://assets.vogue.com/photos/66437c9d87e6542decc508ff/master/w_2560%2Cc_limit/00051-gucci-resort-2025-details-credit-brand.jpg
Image 94 downloaded successfully
Final Image 95 URL: https://assets.vogue.com/photos/664331df34a9d6a41c4f66ba/master/w_2560%2Cc_limit/00026-gucci-resort-2025-credit-brand.jpg
Image 95 downloaded successfully
Final Image 96 URL: https://assets.vogue.com/photos/66437c947498c04cb6a80f05/master/w_2560%2Cc_limit/00044-gucci-resort-2025-details-credit-brand.jpg
Image 96 downloaded successfully
Final Image 97 URL: https://assets.vogue.com/photos/664331d674d2627fa63eb3f3/master/w_2560%2Cc_limit/00017-gucci-resort-2025-credit-brand.jpg
Image 97 downloaded successfully
Final Image 98 URL: https://assets.vogue.com/photos/664331efea2c6e6d9b713920/master/w_2560%2Cc_limit/00040-gucci-resort-2025-credit-brand.jpg
Image 98 downloaded successfully
Final Image 99 URL: https://assets.vogue.com/photos/66437c7ba1ad1be29a88f3fa